In [3]:
import pandas as pd
import numpy as np

DIR = 'C:\\Users\\jesse\\Desktop\\Honors Project\\goodreads_data\\raw\\'

ya_reviews = pd.read_parquet(DIR +'ya_reviews_prepped.parquet')

In [4]:
ya_reviews.head()

,book_id,review_id,rating,review_text,n_votes,n_comments,all_caps,review_text_lemmatized
0,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,24,25,0,i crack and finally pick this up very enjoyabl...
1,23302416,84c0936a0f9868f38e75d2f9a5cb761e,5,I read this book because my fifth grade son wa...,0,0,0,i read this book because my fifth grade son be...
2,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,0,0,0,though the book start out slow and only start ...
3,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",0,0,0,after some sleep i think about allegiant overa...
4,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",0,0,0,this be a move heartbreaking view into a life ...


In [5]:
ya_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2388654 entries, 0 to 2388653
Data columns (total 8 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   book_id                 int32 
 1   review_id               object
 2   rating                  int8  
 3   review_text             object
 4   n_votes                 int16 
 5   n_comments              int16 
 6   all_caps                int32 
 7   review_text_lemmatized  object
dtypes: int16(2), int32(2), int8(1), object(3)
memory usage: 84.3+ MB


### Simple sentiment

In [7]:
test = ya_reviews.head(25)

In [14]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the vader_lexicon
nltk.download('vader_lexicon')

# Instantiate the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Let's assume test['review_text_lemmatized'] is your lemmatized reviews
test_lem_reviews = test['review_text_lemmatized']

# Get the sentiment scores for each review
sentiment_scores = [sia.polarity_scores(review) for review in test_lem_reviews]

# Print the sentiment scores
for score in sentiment_scores:
    print(score)

{'neg': 0.108, 'neu': 0.715, 'pos': 0.177, 'compound': 0.9766}
{'neg': 0.033, 'neu': 0.717, 'pos': 0.25, 'compound': 0.9122}
{'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'compound': 0.5574}
{'neg': 0.052, 'neu': 0.905, 'pos': 0.043, 'compound': -0.7018}
{'neg': 0.059, 'neu': 0.831, 'pos': 0.11, 'compound': 0.7795}
{'neg': 0.052, 'neu': 0.798, 'pos': 0.149, 'compound': 0.9072}
{'neg': 0.045, 'neu': 0.647, 'pos': 0.309, 'compound': 0.9615}
{'neg': 0.0, 'neu': 0.173, 'pos': 0.827, 'compound': 0.6988}
{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.8622}
{'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'compound': 0.6948}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.6919}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.609, 'pos': 0.391, 'compound': 0.91}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.067, 'neu': 0.772, 'pos': 0.161, 'compound': 0.9073}
{'neg': 0.021, 'neu': 0.718, 'pos': 0.

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jesse\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
#assign to column in dataframe
test['sentiment'] = sentiment_scores

C:\Users\jesse\AppData\Local\Temp\ipykernel_36944\856955519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sentiment'] = sentiment_scores


In [16]:
test

,book_id,review_id,rating,review_text,n_votes,n_comments,all_caps,review_text_lemmatized,sentiment
0,2767052,248c011811e945eca861b5c31a549291,5,I cracked and finally picked this up. Very enj...,24,25,0,i crack and finally pick this up very enjoyabl...,"{'neg': 0.108, 'neu': 0.715, 'pos': 0.177, 'co..."
1,23302416,84c0936a0f9868f38e75d2f9a5cb761e,5,I read this book because my fifth grade son wa...,0,0,0,i read this book because my fifth grade son be...,"{'neg': 0.033, 'neu': 0.717, 'pos': 0.25, 'com..."
2,18053080,785c8db878f4009da9741dea51f641da,4,Though the book started out slow and only star...,0,0,0,though the book start out slow and only start ...,"{'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'comp..."
3,17383543,34dc3c45d07e82718b05e73167259aef,2,"*Update - 10/27/13* - After some sleep, I thin...",0,0,0,after some sleep i think about allegiant overa...,"{'neg': 0.052, 'neu': 0.905, 'pos': 0.043, 'co..."
4,16651458,d8d6b590780256fef7ae4a9550fe3e0d,5,"This is a moving, heartbreaking, view into a l...",0,0,0,this be a move heartbreaking view into a life ...,"{'neg': 0.059, 'neu': 0.831, 'pos': 0.11, 'com..."
5,10782699,972ce1267de0213e3032c685386890e6,5,I never thought I would enjoy a zombie books w...,0,0,0,i never think i would enjoy a zombie book with...,"{'neg': 0.052, 'neu': 0.798, 'pos': 0.149, 'co..."
6,7260188,21063e8930ec28bc54e76f932aa30ce1,5,What a great ending to the trilogy!! The secon...,0,0,0,what a great end to the trilogy ! ! the second...,"{'neg': 0.045, 'neu': 0.647, 'pos': 0.309, 'co..."
7,6148028,a19c6c44a40a88c8ff825f40118a9c7c,5,LOVED IT!!,0,0,1,love it ! !,"{'neg': 0.0, 'neu': 0.173, 'pos': 0.827, 'comp..."
8,2767052,c52e231744768e9d7f939d1cbeb87666,5,"Exciting, fun, entertaining! :)",0,0,0,excite fun entertaining !,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound..."
9,13206828,1d5b64fe0408bb5e1824d14631ccb5ee,5,LET'S JUST SAY I AM SO THANKFUL I HAVE THE NEX...,0,0,1,let u just say i be so thankful i have the nex...,"{'neg': 0.0, 'neu': 0.699, 'pos': 0.301, 'comp..."


### TF-IDF Vectorization

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the data
vectorizer.fit(test['review_text_lemmatized'])

# Transform the texts into TF-IDF vectors
tfidf_vectors = vectorizer.transform(test['review_text_lemmatized'])

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define your model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=256))  # input_dim should be the size of your vocabulary
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(10000, activation='softmax'))  # output_dim should be the size of your vocabulary

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
# x_train and y_train should be your preprocessed text data
# Each sample in x_train is a sequence of word indices, and the corresponding sample in y_train is the next word
model.fit(x_train, y_train, epochs=10, batch_size=128)

Word2Vec

from gensim.models import Word2Vec

# Let's assume you have a list of lemmatized texts, each text is a list of words
lemmatized_texts = [['word1', 'word2', 'word3'], ['word4', 'word5', 'word6'], ...]

# Train a Word2Vec model
model = Word2Vec(lemmatized_texts, size=100, window=5, min_count=1, workers=4)

# Get the word vector for a given word
word_vector = model.wv['word1']